#Image Synthesis  

##Written by Gergana Slaveykova s1070004
##Radboud University- B3 Thesis project

In [ ]:
#Mount Google Drive to acess data and storing purposes
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
#Imports
! git clone https://github.com/autonomousvision/stylegan_xl.git
! pip install timm==0.6.13
! pip install ftfy
! pip install einops ninja
! pip install dill
! wget -c "https://s3.eu-central-1.amazonaws.com/avg-projects/stylegan_xl/models/imagenet512.pkl"

import sys
sys.path.append('./stylegan_xl')
import ast
import dnnlib
import legacy
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch
from torch_utils import gen_utils
import torch.nn.functional as F


In [ ]:
#set device
device = torch.device('cuda')

#Load the generator
with dnnlib.util.open_url("imagenet512.pkl") as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore

print(device)

In [ ]:
def reconstruction(latents):
    # Synthesize an image from the latent vector
    img = G.synthesis(latents[None], noise_mode="none")
    # Permute the image dimensions to match the expected format (batch, height, width, channels)
    # Scale pixel values from [-1, 1] to [0, 255] and clamp to valid range [0, 255]
    img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
     # Convert the tensor to a PIL image
    img = Image.fromarray(img[0].cpu().numpy(), 'RGB')
    return img


In [ ]:
#Loading Predicted Latents
w_te = np.load("/content/drive/MyDrive/GOD-NEW-3p/latents/t_te_best.npy")

y_te_p=np.load("/content/drive/MyDrive/z-scored_filtered_my_method/isolationfiltered_ppa_y_te.npy",allow_pickle=True)
y_te_f=np.load("/content/drive/MyDrive/z-scored_filtered_my_method/isolationfiltered_ffa_y_te.npy",allow_pickle=True)
y_te_loc=np.load("/content/drive/MyDrive/z-scored_filtered_my_method/isolationfiltered_loc_y_te.npy",allow_pickle=True)
y_te_v4=np.load("/content/drive/MyDrive/z-scored_filtered_my_method/isolationfiltered_v4_y_te.npy",allow_pickle=True)
y_te_v3=np.load("/content/drive/MyDrive/z-scored_filtered_my_method/isolationfiltered_v3_y_te.npy",allow_pickle=True)
y_te_v2=np.load("/content/drive/MyDrive/z-scored_filtered_my_method/isolationfiltered_v2_y_te.npy",allow_pickle=True)
y_te_v1=np.load("/content/drive/MyDrive/z-scored_filtered_my_method/isolationfiltered_v1_y_te.npy",allow_pickle=True)


In [ ]:
 import os

# List of different latent vector sets
y_te_all = [y_te_p, y_te_f, y_te_loc, y_te_v4, y_te_v3, y_te_v2, y_te_v1]

# Corresponding folder names for each latent vector set
folder=["PPA","FFA","LOC","V4","V3","V2","V1"]

# Iterate over each latent vector set and its corresponding folder name
for ind,data in enumerate(y_te_all):
  print(folder[ind])
  # Repeat the latent vectors 37 times, transpose, and convert to PyTorch tensor
  latents = torch.from_numpy(np.repeat(data[None], 37, axis=0).transpose(1, 0, 2)).to(device)
  latents.shape
  # Create the save directory if it does not exist
  save_dir=f"/content/drive/MyDrive/z-scored_filtered_my_method/IsolationFiltered/{folder[ind]}/"
  os.makedirs(save_dir, exist_ok=True)
  # Iterate over each latent vector in the tensor
  for ind, latent in enumerate(latents):
      # Reconstruct the image from the latent vector
      image = reconstruction(latent)
      file_name = f"image_{ind}.png"  # Define the file name with the desired index
      file_path = os.path.join(save_dir, file_name)  # Construct the full file path
      image.save(file_path)  # Save the image


PPA
FFA
LOC
V4
V3
V2
V1
